### The University of Melbourne, School of Computing and Information Systems
# COMP90086 Computer Vision, 2023 Semester 2

## Assignment 2

**Student Name:**    `TANZID SULTAN`

**Student ID:**     `1430660`

## This iPython notebook is a template which you should use for your Assignment 2 submission. This file should be submitted at the **Assignment 2: Code** link on the LMS.

In addition to this file, you should submit a written report explaining your results at the **Assignment 2: Report** link on the LMS. Please see the assignment specification for details on what must be included in the report for each question.

*Adding proper comments to your code is MANDATORY.*

### 1. CNN training

The following code section will load the dataset from a folder that you specify. You may change the batch sizes and add additional data augmentation steps here if you wish.

In [2]:
from keras.preprocessing.image import ImageDataGenerator

data_dir = 'scene32' # SPECIFY YOUR PATH TO THE DATASET
batch_size = 40

# Data generator for training
# Additional data augmentation may be added here if desired
train_datagen = ImageDataGenerator(
    rescale=1./255)

# Data generator for test
# DO NOT USE DATA AUGMENTATION WITH TEST SET
test_datagen = ImageDataGenerator(
    rescale=1./255)

# Generate training data from 'train' directory
train_data = train_datagen.flow_from_directory(
        data_dir+'/train',
        target_size=(32,32),
        batch_size=batch_size,
        class_mode='categorical')

# Generate vaildation data from 'valid' directory
val_data = test_datagen.flow_from_directory(
        data_dir+'/valid',
        target_size=(32,32),
        batch_size=batch_size,
        class_mode='categorical')

# Generate test data from 'test' directory
test_data = test_datagen.flow_from_directory(
        data_dir+'/test',
        target_size=(32,32),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

# Output list of class names
class_names_and_indices = train_data.class_indices
print(class_names_and_indices)

# Note: To train a model with the dataset loaded above, pass the train and val
# datasets to the fit() function, like so:
# myModel.fit(train_data, validation_data=val_data, ...)

Found 1440 images belonging to 8 classes.
Found 320 images belonging to 8 classes.
Found 320 images belonging to 8 classes.
{'coast': 0, 'forest': 1, 'highway': 2, 'insidecity': 3, 'mountain': 4, 'opencountry': 5, 'street': 6, 'tallbuilding': 7}


In [6]:
# library imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

# make sure GPU is available
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### CNN model according to specs

In [10]:
# build the model
cnn1 = keras.Sequential(
    [
        layers.Input((32,32,3)),                                       # input image shape is 32x32x3  
        layers.Conv2D(16, (3,3), padding='valid', activation='relu'),
        layers.Conv2D(16, (3,3), padding='valid', activation='relu'),
        layers.MaxPooling2D((2,2), strides=(2,2), padding='valid'),
        layers.Flatten(),                                              # unravel the 3d tensor from the previous layer output
        layers.Dense(64, activation='relu'),
        layers.Dense(8, activation='softmax')                          # output layer with 8 neurons, corresponding to the 8 categories
    ]
)


# configure the model
cnn1.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

# display summary
cnn1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 16)        2320      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 3136)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                200768    
                                                                 
 dense_7 (Dense)             (None, 8)                 520       
                                                      

In [11]:
# train the model
history_cnn1 = cnn1.fit(train_data, validation_data=val_data, epochs=50, verbose=2) 

Epoch 1/50


2023-09-01 15:31:59.349482: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-09-01 15:32:01.807374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-09-01 15:32:04.230162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-01 15:32:04.242730: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f86260ad910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-01 15:32:04.242766: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2023-09-01 15:

36/36 - 7s - loss: 1.8882 - accuracy: 0.3056 - val_loss: 1.6711 - val_accuracy: 0.4000 - 7s/epoch - 183ms/step
Epoch 2/50


2023-09-01 15:32:05.804897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


36/36 - 0s - loss: 1.5343 - accuracy: 0.4542 - val_loss: 1.4090 - val_accuracy: 0.5188 - 285ms/epoch - 8ms/step
Epoch 3/50
36/36 - 0s - loss: 1.3021 - accuracy: 0.5562 - val_loss: 1.2388 - val_accuracy: 0.5562 - 279ms/epoch - 8ms/step
Epoch 4/50
36/36 - 0s - loss: 1.1703 - accuracy: 0.6062 - val_loss: 1.1341 - val_accuracy: 0.5906 - 261ms/epoch - 7ms/step
Epoch 5/50
36/36 - 0s - loss: 0.9883 - accuracy: 0.6812 - val_loss: 1.0196 - val_accuracy: 0.6438 - 264ms/epoch - 7ms/step
Epoch 6/50
36/36 - 0s - loss: 0.9024 - accuracy: 0.6868 - val_loss: 1.0155 - val_accuracy: 0.6531 - 256ms/epoch - 7ms/step
Epoch 7/50
36/36 - 0s - loss: 0.8358 - accuracy: 0.7201 - val_loss: 0.9692 - val_accuracy: 0.6719 - 246ms/epoch - 7ms/step
Epoch 8/50
36/36 - 0s - loss: 0.7318 - accuracy: 0.7701 - val_loss: 0.9025 - val_accuracy: 0.6750 - 275ms/epoch - 8ms/step
Epoch 9/50
36/36 - 0s - loss: 0.6845 - accuracy: 0.7743 - val_loss: 0.9649 - val_accuracy: 0.6469 - 277ms/epoch - 8ms/step
Epoch 10/50
36/36 - 0s - lo

### 2. Error anaylsis

### 3. Kernel engineering